# Fetching data using `PandasFetcher`
Translating using pickle files.

In [1]:
import sys
import rics
import id_translation

# Print relevant versions
print(f"{rics.__version__=}")
print(f"{id_translation.__version__=}")
print(f"{sys.version=}")
rics.configure_stuff(rics_level="DEBUG", id_translation_level="DEBUG")
!git log --pretty=oneline --abbrev-commit -1

rics.__version__='3.0.0'
id_translation.__version__='0.3.1.dev1'
sys.version='3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]'
👻 Configured some stuff just the way I like it!
d2093a4 (HEAD, origin/main, origin/HEAD, main) Update help link in TOML files


## Make local Pickle files
We'lll download data from https://datasets.imdbws.com and clean it to make sure all values are given (which means that for actors are dead and titles have stopped airing).

In [2]:
sources = ["name.basics", "title.basics"]

In [3]:
from data import load_imdb

for dataset in sources:
    load_imdb(dataset)

2023-03-25T11:23:35.590 [rics.utility.misc.get_local_or_remote:DEBUG] Local file path: '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/name.basics.tsv.gz'.
2023-03-25T11:23:35.591 [rics.utility.misc.get_local_or_remote:DEBUG] Remote file path: 'https://datasets.imdbws.com/name.basics.tsv.gz'.
2023-03-25T11:23:35.614 [rics.utility.misc.get_local_or_remote:INFO] Fetching data from 'https://datasets.imdbws.com/name.basics.tsv.gz'..


https://datasets.imdbws.com/name.basics.tsv.gz:   0%|          | 0.00/232M [00:00<?, ?iB/s]

2023-03-25T11:23:39.196 [rics.utility.misc.get_local_or_remote:INFO] Local processed file path: '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/clean_and_fix_ids/name.basics.tsv.pkl'.
2023-03-25T11:23:39.197 [rics.utility.misc.get_local_or_remote:INFO] Running clean_and_fix_ids..
2023-03-25T11:23:57.322 [rics.utility.misc.get_local_or_remote:INFO] Serializing processed data to '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/clean_and_fix_ids/name.basics.tsv.pkl'..
2023-03-25T11:23:57.433 [rics.utility.misc.get_local_or_remote:DEBUG] Local file path: '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/title.basics.tsv.gz'.
2023-03-25T11:23:57.434 [rics.utility.misc.get_local_or_remote:DEBUG] Remote file path: 'https://datasets.imdbws.com/title.basics.tsv.gz'.
2023-03-25T11:23:57.434 [rics.utility.misc.get_local_or_remote:INFO] Fetching data from 'https://datasets.imdbws.com/title.basics.tsv.gz'..


https://datasets.imdbws.com/title.basics.tsv.gz:   0%|          | 0.00/162M [00:00<?, ?iB/s]

2023-03-25T11:23:59.305 [rics.utility.misc.get_local_or_remote:INFO] Local processed file path: '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/clean_and_fix_ids/title.basics.tsv.pkl'.
2023-03-25T11:23:59.306 [rics.utility.misc.get_local_or_remote:INFO] Running clean_and_fix_ids..
/home/dev/git/id-translation/jupyterlab/id-translation/data.py:36: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, sep="\t", header=0, engine="c")
2023-03-25T11:24:16.107 [rics.utility.misc.get_local_or_remote:INFO] Serializing processed data to '/home/dev/git/id-translation/jupyterlab/id-translation/data-cache/clean_and_fix_ids/title.basics.tsv.pkl'..


## Create translator from config
Click [here](config.toml) to see the file.

In [4]:
from id_translation import Translator

translator = Translator.from_config("config.toml")
translator

2023-03-25T11:24:16.151 [id_translation.fetching.config-toml.pandas.discovery:DEBUG] Sources initialized: ['name.basics', 'title.basics']


Translator(online=True: fetcher=PandasFetcher(sources=['name.basics', 'title.basics']))

In [5]:
tmap = translator.store().cache

2023-03-25T11:24:16.262 [id_translation.fetching.config-toml:DEBUG] Begin wanted-to-actual placeholder mapping of placeholders={'id', 'to', 'from', 'name'} to actual placeholders={'nconst', 'int_id_nconst', 'knownForTitles', 'deathYear', 'primaryName', 'primaryProfession', 'birthYear'} for source='name.basics'.
2023-03-25T11:24:16.263 [id_translation.mapping.placeholders.config-toml:DEBUG] Begin computing match scores in context='name.basics' for ['id', 'to', 'from', 'name']x['nconst', 'int_id_nconst', 'knownForTitles', 'deathYear', 'primaryName', 'primaryProfession', 'birthYear'] using HeuristicScore([force_lower_case()] -> AbstractFetcher.default_score_function).
2023-03-25T11:24:16.264 [id_translation.mapping.placeholders.config-toml:DEBUG] All values mapped by overrides. Applied 2 overrides, and found 4 matches={'id': 'nconst', 'to': 'deathYear', 'from': 'birthYear', 'name': 'primaryName'} in the given values=['id', 'to', 'from', 'name'].
2023-03-25T11:24:16.266 [id_translation.fet

In [6]:
for source in tmap:
    translations = tmap[source]
    print(f"Translations for {source=};")
    for i, (idx, translation) in enumerate(tmap[source].items()):
        print(f"    {repr(idx)} -> {repr(translation)}")
        if i == 2:
            break

Translations for source='title.basics';
    'tt0035803' -> 'tt0035803:The German Weekly Review (original: Die Deutsche Wochenschau) *1940†1945'
    'tt0038276' -> 'tt0038276:You Are an Artist (original: You Are an Artist) *1946†1955'
    'tt0039120' -> 'tt0039120:Americana (original: Americana) *1947†1949'
Translations for source='name.basics';
    'nm0000001' -> 'nm0000001:Fred Astaire *1899†1987'
    'nm0000002' -> 'nm0000002:Lauren Bacall *1924†2014'
    'nm0000004' -> 'nm0000004:John Belushi *1949†1982'
